In [10]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import mutual_info_regression


In [6]:
all_data = pd.read_csv('../data/all_data.csv').drop('Unnamed: 0', axis=1)

In [8]:
all_data.columns

Index(['date', 'aluminum', 'brent', 'cbr_key_rate', 'eur_rub', 'moex_index',
       'nickel', 'rtsi', 'usd_rub', 'su26218', 'su26221', 'su26222', 'su26224',
       'su26230', 'gazp', 'gmkn', 'lkoh', 'magn', 'mgnt', 'moex', 'rosn',
       'rual', 'sber', 'vtbr'],
      dtype='object')

In [14]:
risk_factors_dtc ={
'sber': [], #сбер
'vtbr': [], #втб
'rual': [], #русал
'rosn': ['brent', 'eur_rub', 'usd_rub'], #роснефть
'moex': [], #мосбиржа
'mgnt': [], #магнит
'magn': [], #ММК
'lkoh': [], #лукойл
'gmkn': [], #гмк норникель
'gazp': [] #газпром

}

In [63]:
def calc_mutual_info(y, X):
    """
    Selects top N features by Mutual Information with target
    """
    # determine the mutual information
    mutual_info = mutual_info_regression(X.fillna(0), y)
    mutual_info = pd.Series(mutual_info)
    mutual_info.index = X.columns
    return list(mutual_info.sort_values(ascending=False).index)

In [78]:
def select_risk_factors(factor, risk_factors_list):
    """
    Функция принимает на вход колонку-название актива и список его возможных риск-факторов из словаря 
    Отсеиваются факторы с корреляцией между фактором и таргетом < 0.05
    Смотрится взаимная корреляция факторов, если она превышает 0.9, то оставляется один фактор с наибольшей MI с таргетом
    """
    target = all_data[factor].values
    risk_factors = all_data[risk_factors_list]
    drop_list_idx = []
    corr_list = []
    #считаем обычную корреляцию с таргетом
    for i in range(len(risk_factors.columns)):
        corr_with_target = np.corrcoef(x = risk_factors[risk_factors_list[i]].values, y = target)[0][1]
        corr_list.append(corr_with_target)
        if corr_with_target < 0.05: #дропаем, если меньше 0.05
            drop_list_idx.append(i)
    
    #ранжируем фичи по MI с таргетом
    top_mutual_information_list = calc_mutual_info(target, risk_factors)

    #смотрим взаимную корреляцию фичей
    mutual_correlation = np.corrcoef(x = risk_factors, rowvar=False)
    a = np.where(mutual_correlation >= 0.9)[0]
    b = np.where(mutual_correlation >= 0.9)[1]
    correlated_features = []
    for i in range(len(a)):
        if a[i]!= b[i]:
            correlated_features.append(tuple(sorted([a[i], b[i]])))
    correlated_features = set(correlated_features) #тут сет пар индексов скоррелированных фичей

    for pair in correlated_features: #смотрим, какая из фичей из пары на каком месте в ранжированном списке MI
        mi_1 =np.where(np.array(top_mutual_information_list) == risk_factors.columns[pair[0]]) 
        mi_2 =np.where(np.array(top_mutual_information_list) == risk_factors.columns[pair[1]])

        if mi_1 > mi_2: #если первый признак из пары менее связан с таргетом (дальше от начала списка MI)
            drop_list_idx.append(pair[0]) #дропаем первый признак
        else:
            drop_list_idx.append(pair[1])
    
    drop_list_names = list(risk_factors[risk_factors.columns[drop_list_idx]].columns)
    print(drop_list_names)
    return all_data[[factor] + risk_factors_list].drop(drop_list_names, axis=1)
        

In [79]:
select_risk_factors('rosn', risk_factors_dtc['rosn'])

['usd_rub']


,rosn,brent,eur_rub
0,473.50,65.02,68.0555
1,481.15,64.23,68.0450
2,475.50,64.42,67.8162
3,474.50,64.00,68.4213
4,480.95,64.62,68.3747
...,...,...,...
884,577.95,79.39,99.0480
885,583.20,79.07,100.5113
886,591.45,79.54,101.1178
887,593.95,77.15,101.3451
